In [1]:
from langchain_openai import OpenAIEmbeddings
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
import os
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from IPython.display import display

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up connection to Neo4j

In [2]:
neo4j_password = getpass()

 ········


In [180]:
openai_api_key = getpass()

 ········


In [3]:
neo4j_uri = "neo4j+s://2fe3bf28.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Find themes with common stems

In [303]:
nltk.download("punkt")

ps = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nathansmith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [304]:
theme_df = gds.run_cypher("""MATCH (t:Theme) RETURN t.description AS description""")

In [305]:
theme_df['stem'] = theme_df['description'].map(lambda x: ps.stem(x))

In [306]:
grouped_themes = theme_df.groupby('stem').apply(np.unique, include_groups=False)

In [307]:
shared_stems = grouped_themes[grouped_themes.map(len)>1]
shared_stems_df = shared_stems.reset_index()
shared_stems_df.columns = ['stem', 'descriptions']
shared_stems_df.shape

(2646, 2)

In [308]:
shared_stems_df.sample(10)

,stem,descriptions
2049,scaveng,"[Scavenger, scavengers]"
314,carpent,"[Carpenters, carpenter]"
2050,sceneri,"[sceneries, scenery]"
295,camera fe,"[camera feed, camera feeds]"
680,doppelgäng,"[Doppelgängers, doppelgänger]"
599,denial,"[denial, denials]"
259,broken hom,"[broken home, broken homes]"
2454,twist,"[twist, twisted, twists]"
179,bell,"[Bell, Bells]"
322,caterpillar,"[Caterpillars, caterpillar]"


## Send stems shared by multiple themes to Neo4j

In [310]:
gds.run_cypher("""CREATE CONSTRAINT stem_node_key IF NOT EXISTS FOR (s:Stem) REQUIRE s.stem IS NODE KEY""")

""


In [311]:
gds.run_cypher("""UNWIND $data AS row
MERGE (s:Stem {stem:row['stem']})
WITH row, s
UNWIND row['descriptions'] AS description
MATCH (t:Theme {description:description})
MERGE (t)-[:HAS_STEM]->(s)
SET s.descriptions = row['descriptions']
RETURN count(*) AS stemmedCount""",
              {"data": shared_stems_df.to_dict("records")})

,stemmedCount
0,5767


## Aggregate theme vectors for themes that share a stem

In [312]:
shared_stem_df = gds.run_cypher("""
MATCH (s:Stem)<-[:HAS_STEM]-(t)
RETURN s.stem AS stem, collect(t.embedding) AS embeddings
""")

In [313]:
def get_mean_vector(row):
    mean_vector = np.array(row['embeddings']).mean(axis=0).tolist()
    return mean_vector

In [314]:
shared_stem_df['mean_embedding'] = shared_stem_df.apply(get_mean_vector, axis=1)

In [315]:
gds.run_cypher("""
    UNWIND $data AS row
    MATCH (s:Stem {stem: row['stem']})
    CALL db.create.setNodeVectorProperty(s, "meanEmbedding", row['mean_embedding'])""",
                   {"data":shared_stem_df[['stem', 'mean_embedding']].to_dict("records")})


""


# Create projection
Include stems and individual themes that do not share a stem with another theme.

Use the theme embedding and the aggregated embedding for stems.

In [487]:
g_themes, result = gds.graph.cypher.project("""
MATCH (t:Stem|Theme)
WHERE NOT EXISTS {(t)-[:HAS_STEM]->()}
RETURN gds.graph.project("themes", 
t, null,
{sourceNodeProperties: {embedding: CASE WHEN t:Stem then t.meanEmbedding ELSE t.embedding END},
targetNodeProperties: null}
)""")

Failed to write data to connection ResolvedIPv4Address(('34.28.32.244', 7687)) (ResolvedIPv4Address(('34.28.32.244', 7687)))
Failed to write data to connection IPv4Address(('2fe3bf28.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.28.32.244', 7687)))


In [488]:
result

{'relationshipCount': 0,
 'graphName': 'themes',
 'query': '\nMATCH (t:Stem|Theme)\nWHERE NOT EXISTS {(t)-[:HAS_STEM]->()}\nRETURN gds.graph.project(******, \nt, ******,\n{sourceNodeProperties: {embedding: CASE WHEN t:Stem then t.meanEmbedding ELSE t.embedding END},\ntargetNodeProperties: ******}\n)',
 'projectMillis': 9561,
 'configuration': {'readConcurrency': 4,
  'undirectedRelationshipTypes': [],
  'jobId': 'd46f66fb-9cb8-4608-99b7-0a5dd3b1cbdf',
  'logProgress': True,
  'query': '\nMATCH (t:Stem|Theme)\nWHERE NOT EXISTS {(t)-[:HAS_STEM]->()}\nRETURN gds.graph.project(******, \nt, ******,\n{sourceNodeProperties: {embedding: CASE WHEN t:Stem then t.meanEmbedding ELSE t.embedding END},\ntargetNodeProperties: ******}\n)',
  'inverseIndexedRelationshipTypes': [],
  'creationTime': neo4j.time.DateTime(2024, 3, 22, 17, 56, 57, 945894861, tzinfo=<UTC>)},
 'nodeCount': 36970}

# Test some theme pairs to find a score above which you think themes are probably duplicated
Get a sense of the range of similarity values for top 5 nearest neighbors.

Note that KNN returns cosine similarity normalized to the range [0, 1]. 

If you run gds.similarity.cosine() you get similarity in the range [-1,1]

In [322]:
knn_stats = gds.knn.stats(g_themes, topK=5, nodeProperties = "embedding")

In [323]:
knn_stats['similarityDistribution']

{'min': 0.6269302368164062,
 'p5': 0.6885490417480469,
 'max': 0.9965553283691406,
 'p99': 0.9251556396484375,
 'p1': 0.6686859130859375,
 'p10': 0.7019996643066406,
 'p90': 0.86669921875,
 'p50': 0.7789230346679688,
 'p25': 0.7316360473632812,
 'p75': 0.8288040161132812,
 'p95': 0.8873481750488281,
 'mean': 0.7821673368743409,
 'p100': 0.9965553283691406,
 'stdDev': 0.06246581380926736}

## Given a theme, check to see how many neighbors are duplicates and the similarity score for the first neighbor you would consider to be a different theme.

db.index.vector.queryNodes() returns cosine similarity normalized to the range [0, 1]

In [331]:
top_k_df = gds.run_cypher("""
MATCH (t1:Theme) WHERE t1.description IN $themeList
CALL db.index.vector.queryNodes("theme_vectors", $k + 1, t1.embedding) YIELD node, score
WITH t1, node AS t2, score
WHERE t1 <> t2
RETURN t1.description AS theme1,
t2.description AS theme2,
score
ORDER BY theme1, score DESC""",
               {"k": 20,
                "themeList": ['friend', 'dance floor', 'Fast-food burger', 'guilt', 'antics', 
                              'ants', 'underwater', 'Africa', 'ChatGPT', 'coming-of-age']})

In [255]:
top_k_df[top_k_df['theme1'] == 'friend']
# top 4 cutoff 0.811518

,theme1,theme2,score
120,friend,Friendly,0.825079
121,friend,friend group,0.823099
122,friend,friends,0.822041
123,friend,friendship,0.817366
124,friend,dream wedding,0.811518
125,friend,love,0.801542
126,friend,partner,0.799027
127,friend,family,0.797425
128,friend,dream mentor,0.795746
129,friend,friendships,0.792070


In [256]:
top_k_df[top_k_df['theme1'] == 'dance floor']
# top 2 cutoff 0.876044

,theme1,theme2,score
100,dance floor,Dancefloor,0.907340
101,dance floor,dance party,0.876044
102,dance floor,dance class,0.847459
103,dance floor,dance time,0.845335
104,dance floor,dance routines,0.844204
105,dance floor,dance,0.838422
106,dance floor,go-go dancer,0.837511
107,dance floor,dancing,0.837206
108,dance floor,dance studio,0.835570
109,dance floor,Step dancing,0.829465


In [257]:
top_k_df[top_k_df['theme1'] == 'Fast-food burger']
# top 4 cutoff 0.807412

,theme1,theme2,score
40,Fast-food burger,fast food restaurant,0.854380
41,Fast-food burger,fast food,0.845039
42,Fast-food burger,burgers,0.825347
43,Fast-food burger,Burger Baby,0.810610
44,Fast-food burger,Vegan burgers,0.807412
45,Fast-food burger,Good Burger,0.800079
46,Fast-food burger,Burger van,0.780409
47,Fast-food burger,French fries,0.754924
48,Fast-food burger,burger,0.753647
49,Fast-food burger,street food,0.746388


In [258]:
top_k_df[top_k_df['theme1'] == 'guilt']
# top 6 cutoff 0.793735

,theme1,theme2,score
140,guilt,Guilt and fear,0.822317
141,guilt,Guilt and paranoia,0.816934
142,guilt,blame,0.814373
143,guilt,shame,0.802213
144,guilt,self-blame,0.795776
145,guilt,remorse,0.795585
146,guilt,culprit,0.793735
147,guilt,accused,0.788248
148,guilt,conscience,0.778244
149,guilt,responsibility,0.768354


In [259]:
top_k_df[top_k_df['theme1'] == 'antics']
# top 0 cutoff 0.854217

,theme1,theme2,score


In [330]:
top_k_df[top_k_df['theme1'] == 'ants']
# top 0 cutoff 0.854217

,theme1,theme2,score
80,ants,antics,0.854217
81,ants,agents,0.851598
82,ants,rats,0.833571
83,ants,names,0.832501
84,ants,guns,0.828042
85,ants,webs,0.812752
86,ants,ads,0.807464
87,ants,pies,0.805520
88,ants,target,0.804613
89,ants,stocks,0.802675


In [261]:
top_k_df[top_k_df['theme1'] == 'underwater']
# top 4 cutoff 0.880904

,theme1,theme2,score
160,underwater,undersea,0.946337
161,underwater,subaquatic,0.905288
162,underwater,underwater world,0.896511
163,underwater,undersea world,0.881089
164,underwater,underwater music,0.880904
165,underwater,underwater adventure,0.875346
166,underwater,underwater exploration,0.873639
167,underwater,deep sea,0.857809
168,underwater,underwater ballet,0.847281
169,underwater,submarine,0.846147


In [262]:
top_k_df[top_k_df['theme1'] == 'Africa']
# top 0 cutoff 0.829958

,theme1,theme2,score
0,Africa,Asia,0.829958
1,Africa,Europe,0.793405
2,Africa,African,0.792190
3,Africa,Atlantic,0.775041
4,Africa,America,0.765479
5,Africa,Asian,0.750807
6,Africa,African Union,0.750721
7,Africa,Pan-African,0.749782
8,Africa,African nation,0.748957
9,Africa,Horn of Africa,0.748596


In [263]:
top_k_df[top_k_df['theme1'] == 'ChatGPT']
# top 2 cutoff 0.718462

,theme1,theme2,score
20,ChatGPT,Chat-GPT,0.951617
21,ChatGPT,GPT-3,0.816252
22,ChatGPT,G,0.718462
23,ChatGPT,CG,0.715920
24,ChatGPT,Hangouts,0.714553
25,ChatGPT,GeGeGe,0.713404
26,ChatGPT,CGI,0.706873
27,ChatGPT,GDR,0.705540
28,ChatGPT,genesis,0.704529
29,ChatGPT,SMGP,0.703914


In [334]:
top_k_df[top_k_df['theme1'] == 'coming-of-age']
# top 5 cutoff 0.791164

,theme1,theme2,score
100,coming-of-age,coming of age,0.980575
101,coming-of-age,adolescence,0.819819
102,coming-of-age,Growing-up,0.819564
103,coming-of-age,puberty,0.802797
104,coming-of-age,growing up,0.799015
105,coming-of-age,post-adolescence,0.791164
106,coming-of-age,boyhood,0.790140
107,coming-of-age,lost youth,0.770246
108,coming-of-age,youthful adventure,0.765915
109,coming-of-age,Transition to adulthood,0.763636


# Based on the exploration above run KNN with a similarityCutoff you choose

In [490]:
similarity_cutoff = 0.811518

knn_result = gds.knn.mutate(g_themes,
                            mutateRelationshipType = "IS_SIMILAR",
                            mutateProperty = "similarity",
                            nodeProperties = "embedding",
                            topK = 4,
                            sampleRate = 1.0,
                            perturbationRate = 0.5,
                            deltaThreshold = 0.00001,
                            similarityCutoff = similarity_cutoff)

## Find themes that discovered no neighbors above the similarity threshold you selected

In [ ]:
no_sim_df = gds.run_cypher("""
CALL gds.graph.relationships.stream("themes", ["IS_SIMILAR"]) yield sourceNodeId, targetNodeId
UNWIND [gds.util.asNode(sourceNodeId), gds.util.asNode(targetNodeId)] AS nodeWithSim
WITH collect(DISTINCT nodeWithSim) AS nodesWithSim
MATCH (t:Theme) WHERE NOT t IN nodesWithSim
AND NOT EXISTS {(t)-[:HAS_STEM]->()}
RETURN t.description AS description, count{ (t)<-[:HAS_THEME]-() } AS movieCount
ORDER BY movieCount desc""")

It's weird that "coming of age" shows up here, when we know that it has neghbor "coming-of-age" with score 0.980575 from the analysis above. KNN is using approximate nearest neighbors, and even with a sample rate of 1.0 it can miss some possible matches.

In [405]:
no_sim_df

,description,movieCount
0,coming of age,71
1,poverty,55
2,London,54
3,paranoia,52
4,wrestling,51
...,...,...
16587,sauce,1
16588,saudade,1
16589,sausage rolls,1
16590,saving history,1


In [406]:
no_sim_df[no_sim_df['movieCount'] > 1]['description'].count()

2174

## Examine theme pairs for high degree nodes
For Stem nodes, turn their list of descriptions into a comma separated string before exporting.

In [407]:
similarity_result = gds.run_cypher("""
    CALL gds.graph.relationshipProperty.stream("themes", "similarity", ["IS_SIMILAR"])
    YIELD sourceNodeId, targetNodeId, propertyValue AS similarity
    WITH gds.util.asNode(sourceNodeId) AS source, gds.util.asNode(targetNodeId) AS target, similarity
    RETURN coalesce(source.description, reduce(s = source.descriptions[0], d in source.descriptions[1..] | s + ", " + d)) AS source, 
    coalesce(target.description, reduce(s = target.descriptions[0], d in target.descriptions[1..] | s + ", " + d)) AS target, similarity
    ORDER BY similarity""")
                    

What are the nodes with the most inbound IS_SIMILAR relationships?

In [408]:
similarity_result.groupby("target")['source'].count().sort_values(ascending=False).head(20)

target
Humanism, Humanities, human, humanity, humanization, humans    39
haunted house, haunted houses                                  39
dark place                                                     37
acting dreams                                                  37
dream, dreaming, dreams                                        36
Musically, music, musical, musicals                            35
dance, dances, dancing                                         35
hidden secrets                                                 34
Rural community, rural communities                             33
LGBTQ+                                                         32
heartfelt stories                                              30
Wild journey                                                   29
Hauntings, haunt, haunted, haunting                            29
ordinary life                                                  29
human condition                                                28
eld

In [409]:
similarity_result[similarity_result['target']=="haunted house, haunted houses"]

,source,target,similarity
8892,school haunting,"haunted house, haunted houses",0.823204
10899,haunted letterbox,"haunted house, haunted houses",0.825859
12158,haunted promise,"haunted house, haunted houses",0.827624
12334,haunted doll,"haunted house, haunted houses",0.827866
15657,haunted hairpin,"haunted house, haunted houses",0.832387
17512,haunted coffee,"haunted house, haunted houses",0.835167
18009,haunted childhood,"haunted house, haunted houses",0.835853
23221,haunted phone,"haunted house, haunted houses",0.843519
24694,haunted shadows,"haunted house, haunted houses",0.845714
28595,haunted photos,"haunted house, haunted houses",0.851900


In [410]:
similarity_result[similarity_result['target']=="Humanism, Humanities, human, humanity, humanization, humans"]

,source,target,similarity
786,Humanity's rebirth,"Humanism, Humanities, human, humanity, humanization, humans",0.812533
3167,humanized image,"Humanism, Humanities, human, humanity, humanization, humans",0.815523
4104,human bond,"Humanism, Humanities, human, humanity, humanization, humans",0.816853
8113,humanity's anxiety,"Humanism, Humanities, human, humanity, humanization, humans",0.822183
9095,humanity's resistance,"Humanism, Humanities, human, humanity, humanization, humans",0.823478
9301,human vision,"Humanism, Humanities, human, humanity, humanization, humans",0.823740
10864,humanity's brutality,"Humanism, Humanities, human, humanity, humanization, humans",0.825814
12325,post-human,"Humanism, Humanities, human, humanity, humanization, humans",0.827854
12429,Humano,"Humanism, Humanities, human, humanity, humanization, humans",0.827981
12960,Humanity's future,"Humanism, Humanities, human, humanity, humanization, humans",0.828705


In [411]:
similarity_result[similarity_result['target']=="Christmas magic"]

,source,target,similarity
1480,Christmas sweaters,Christmas magic,0.813443
5602,spicy Christmas,Christmas magic,0.818920
7388,Christmas curse,Christmas magic,0.821274
20353,White Christmas,Christmas magic,0.839211
21500,Christmas World,Christmas magic,0.840920
34402,Christmas story,Christmas magic,0.862150
38776,Christmas lights,Christmas magic,0.870777
38860,Christmas Variety,Christmas magic,0.870974
40547,Christmas memories,Christmas magic,0.874529
40860,Xmas,Christmas magic,0.875206


In [412]:
similarity_result[similarity_result['target']=="documentary video"]

,source,target,similarity
2144,music documentary,documentary video,0.814246
2290,DIY Documentary,documentary video,0.814440
8422,generative documentary,documentary video,0.822653
14119,revealing documentary,documentary video,0.830216
17467,social documentary,documentary video,0.835099
22154,documentary confession,documentary video,0.841925
28017,Dogumentary,documentary video,0.850870
28775,Political documentary,documentary video,0.852186
35271,fake documentary,documentary video,0.863661
36881,animated documentary,documentary video,0.866787


# Write weakly connected components with threshold at your KNN similarityCutoff or a higher value

In [413]:
gds.wcc.write(g_themes,
              writeProperty = "wccId",
              relationshipTypes = ["IS_SIMILAR"],
              relationshipWeightProperty = "similarity",
              threshold = 0.875,
              minComponentSize=2)

writeMillis                                                                                                                                                                                                                                                                                                                                                                   107
nodePropertiesWritten                                                                                                                                                                                                                                                                                                                                                        7630
componentCount                                                                                                                                                                                                                                                      

## Examine the largest communities at the KNN value you selected

In [414]:
pd.set_option('display.max_colwidth', None)

In [415]:
gds.run_cypher("""
MATCH (t:Theme)
WHERE t.wccId IS NOT NULL
WITH t.wccId AS id, collect(t.description) AS themes
RETURN id, size(themes) AS themeCount, themes
ORDER BY themeCount DESC
LIMIT 30""")

,id,themeCount,themes
0,369,207,"[Accidental discovery, Ancient lore, Blurring reality, Buried, Creepy Mansion, Darker truth, Disturbing presence, Enchanted forest, Enchanted garden, Enigmatic identity, Enigmatic origin, Enigmatic strangers, Fairy-tale land, Fairytale world, Fiction blurring reality, Fiction reality, Folktale, Forbidden passion, Green fairies, Haunted campus, Haunted carriage, Haunted chapel, Haunted cinema, Haunted hill, Haunted home, Haunted manor, Haunted market, Haunted mask, Haunted memories, Haunted palace, Haunted party, Haunted places, Haunted temple, Haunted theme park, Haunted train, Haunted well, Haunting visions, Hidden desire, Invisible realm, Monster mask, Mysterious Guy, Mysterious old man, Mysterious realm, Mysterious secrets, Mysterious woods, Mystical beast, Mystical beings, Mystical realm, Mythical being, Mythos, Past ghosts, Reality blurred, Secret Mansion, Secret Unravels, Secret passion, Secret uncovered, Secrets unraveling, Shadowy Legacy, Shadowy figure, Shady company, Shady organization, Startling revelations, Supernatural visions, Uncovered secrets, Unhaunted house, Winter fairy tale, air fairies, ancient mythology, another realm, blurred reality, buried alive, buried body, buried dreams, buried history, buried skeleton, buried truth, buried wish, concealed truth, concealing truth, dark history, dark truth, disembodied spirit, dreams vs reality, enchanted world, enigmatic, enigmatic figure, enigmatic past, enigmatic woman, fairy-tale, fairytale characters, fantastical world, fantasy land, fantasy reality, fantasy realm, fantasy world, fiction vs reality, folk tale, folklore, forbidden affair, forbidden attraction, ...]"
1,357,138,"[Absent mother, Divorced father, Elderly daughter, Elderly father, Elderly spinster, Elderly widow, Father's passing, Grieving widow, Loss of loved one, Missing sibling, Mother's despair, Motherly love, Motherly protection, Motherly role, Mothers' support, Parent conflict, Parental conflict, Parental struggles, Parenting challenges, Sibling conflict, Sibling protection, Single parenthood, Stepsiblings, Struggling mother, Widow's obsession, Widow's plight, Widowed father, Widowhood, absent father, aging parents, ailing father, ailing mother, birth mother, brother and sister, brother sister, brother-in-law, brotherhood, brotherly love, brothers and sisters, daughter killed, daughter's death, daughter's life, daughter's story, dead brother, dead sister, deceased father, deceased mother, deceased sister, deceased wife, divorced mother, divorced parents, dying father, elderly, elderly care, elderly gentleman, elderly home, elderly lady, elderly life, elderly man, elderly mother, elderly parent, elderly woman, elderly women, familial bonds, family grief, family loss, family tragedy, father's absence, father's death, father's illness, grieving father, grieving mother, heartbroken widow, kill brother, late father, late husband, loss of father, loss of mother, lost brother, lost mother, maternal absence, maternal influence, maternal love, missing brother, missing father, missing husband, missing mom, missing mother, missing sister, missing spouse, missing wife, mother's, mother's absence, mother's death, mother's departure, mother's grief, mother's house, mother's illness, mother's influence, mother's life, ...]"
2,570,115,"[Broken relationships, Brotherly bonds, Community tradition, DIE, Estranged friend, Estranged sister, Family bond, Missing lover, Mom, Mothers and daughters, Past love, Past lovers, Present lovers, Relationship conflicts, Relationship tension, Relationship tensions, Sibling relationships, Traditional values, Widow's struggles, broken relationship, brother, brotherly bond, brothers, community traditions, couple, couples, cultural tradition, cultural traditions, dying, dying man, estranged, estranged brother, estranged brothers, estranged friends, estranged relationship, estranged relationships, estranged sisters, estrangement, ex-lover, ex-lovers, 

# Create undirected graph for Leiden

The largest WCC communities are too big. For example, "enchanted forest" and "buried body" do not belog in the same community. We'll use Leiden to make smaller comminities.

Leiden requires undirected relationships, so convert the directed IS_SIMILAR relationship to an undirected relationship called "UNDIRECTED_SIMILAR".

In [491]:
gds.run_cypher("""
call gds.graph.relationships.toUndirected("themes", {relationshipType: "IS_SIMILAR", mutateRelationshipType: "UNDIRECTED_SIMILAR", aggregation: "MAX"})""")

,inputRelationships,relationshipsWritten,mutateMillis,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,53686,71200,0,0,0,97,"{'aggregation': 'MAX', 'relationshipType': 'IS_SIMILAR', 'jobId': '7644d61d-1bc9-45ed-8df8-4a438c62b76c', 'logProgress': True, 'concurrency': 4, 'sudo': False, 'mutateRelationshipType': 'UNDIRECTED_SIMILAR'}"


## Transform relationship weights
Because of our similarity cutoff in KNN, all of our relationship scores are between 0.811518 and 1.0. Apply a transformation to spread them out between 0.0 and 1.0.

In [492]:
gds.set_database("neo4j")

In [493]:
gds.run_cypher("""
CALL gds.graph.relationshipProperties.stream($graphName, ["similarity"], ["UNDIRECTED_SIMILAR"])
YIELD sourceNodeId, targetNodeId, propertyValue
WITH sourceNodeId AS source, 
targetNodeId AS target,
max(propertyValue) AS score
WITH gds.graph.project("reweight_themes",
source,
target,
{relationshipType: "IS_SIMILAR",
relationshipProperties: {similarity: (score-$similarityCutoff)/(1-$similarityCutoff)}},
{undirectedRelationshipTypes: ["IS_SIMILAR"]}) AS g
RETURN g.graphName AS graphName, g.relationshipCount AS relationshipCount, g.nodeCount AS nodeCount""",
              {"graphName": g_themes.name(), "similarityCutoff": similarity_cutoff})

,graphName,relationshipCount,nodeCount
0,reweight_themes,142400,19473


In [494]:
g_themes2 = gds.graph.get("reweight_themes")

# Test Leiden with different values of gamma
This function will run the Leiden algorithm with a value of gamma that you provide. It will return the size of the biggest Leiden communities and the themes in those communities. It will also check for the smallest communities that contain only one document.

In [500]:
def test_leiden_gamma(gamma):
    property_name = f"leidenGamma{gamma}"
    try:
        gds.graph.nodeProperties.drop(g_themes2, property_name)
    except:
        pass
    leiden_result = gds.leiden.mutate(g_themes2,
                      mutateProperty = property_name,
                      relationshipTypes = ["IS_SIMILAR"],
                      relationshipWeightProperty = "similarity",
                      gamma=gamma)
    biggest_communities = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_themes", $propName)
        YIELD nodeId, propertyValue
        WITH propertyValue, collect(gds.util.asNode(nodeId)) AS nodeList, count(*) as themeCount
        ORDER BY themeCount desc
        LIMIT 20
        RETURN themeCount, [n in nodeList | coalesce(n.description, n.descriptions)] AS themes""",
                                        {"propName": property_name})
    stats_df = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_themes", "leidenGamma1.0")
        YIELD nodeId, propertyValue
        WITH gds.util.asNode(nodeId) as n, propertyValue AS leidenId
        WITH leidenId, collect(n) AS nodeList
        WHERE size(nodeList) = 1
        WITH leidenId, nodeList[0] AS n
        WITH leidenId,
        COUNT{MATCH (n)<-[:HAS_THEME]-(d) RETURN DISTINCT d} AS themeCount,
        COUNT{MATCH (n)<-[:HAS_STEM]-()<-[:HAS_THEME]-(d) RETURN DISTINCT d} AS stemCount
        WHERE themeCount + stemCount = 1
        RETURN count(*) AS singleDocCommunities""",
                                           {"propName": property_name})
    stats_df.loc[0,'community_count'] = leiden_result['communityCount']
    stats_df.loc[0, 'max_community_size'] = leiden_result['communityDistribution']['max']
    stats_df.loc[0, 'mean_community_size'] = leiden_result['communityDistribution']['mean']
    stats_df.loc[0, 'median_community_size'] = leiden_result['communityDistribution']['p50']
    stats_df.loc[0, 'gamma'] = gamma
    return {"biggest_communities": biggest_communities, "stats": stats_df}
                                           


In [504]:
gamma_tests = {gamma: test_leiden_gamma(gamma) for gamma in [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0, 1024.0]}

In [509]:
stats_df = pd.concat([r['stats'] for r in [gamma_tests[key] for key in gamma_tests.keys()]])
stats_df[['gamma', 'singleDocCommunities', 'community_count', 'max_community_size', 'mean_community_size', 'median_community_size']]

,gamma,singleDocCommunities,community_count,max_community_size,mean_community_size,median_community_size
0,1.0,0,1644.0,430.0,11.844891,2.0
0,2.0,0,1705.0,282.0,11.421114,2.0
0,4.0,0,1792.0,218.0,10.866629,2.0
0,8.0,0,1910.0,145.0,10.195288,2.0
0,16.0,0,2071.0,134.0,9.402704,2.0
0,32.0,0,2254.0,71.0,8.639308,3.0
0,64.0,0,2524.0,53.0,7.715135,3.0
0,128.0,0,2871.0,39.0,6.782654,3.0
0,256.0,0,3369.0,27.0,5.780053,4.0
0,512.0,0,4152.0,20.0,4.690029,4.0


In [511]:
for gamma, result in gamma_tests.items():
    print(f"Gamma value: {gamma}")
    display(result['biggest_communities'].head())

Gamma value: 1.0


,themeCount,themes
0,430,"[strange phenomenon, phenomenon, strange artifact, strange phenomena, Mysterious phenomenon, Bizarre phenomena, phenomena, strange force, strange performance, [strange event, strange events], dark stories, Scary Tales, Gripping tales, terrifying tales, Killer tales, haunted tales, [twisted tale, twisted tales], strange man, strange woman, strange couple, Strange man's house, weird guy, strange owner, strange figure, strange proposal, strange person, strange bird, strange, bleak stories, Legendary tales, supernatural tales, Halloween disappearance, Natural phenomenon, enigmatic, enigmatic key, enigmatic castle, enigmatic figure, enigmatic event, enigmatic patron, enigmatic feelings, enigmatic plot, enigmatic woman, enigmatic past, [Elusiveness, elusive], [Enigmas, enigma], mysterious woman, mystery woman, mysterious man, mysterious women, mysterious object, mysterious body, mysterious entity, mysterious ship, Strange abilities, Strange happenings, Strange situations, [Strange occurrence, strange occurrences], charisma, [charm, charming], Uncommon love, unconventional love, Unusual love, Uncommon name, unusual help, Unusual occurrence, Unusual Case, unusual solution, [unusual method, unusual methods], strange object, [stranger, strangers], Absurd encounters, surreal encounters, [strange encounter, strange encounters], Quirky acquaintance, Eccentric stranger, unconventional theories, Unconventional, unconventional teaching, unorthodox science, unconventional choices, [Unconventional relationships, unconventional relationship], Unconventional life, terrible act, Terrible tragedy, Heinous act, terrible situation, Horrific crime, colorless world, Colourless, wild creatures, native animals, hidden creatures, saving creature, creatures flee, invisible creatures, forest creatures, wildlife, [creature, creatures], Freaky world, Frightening world, ...]"
1,382,"[patriarch, Patriarchal norms, matriarch, mother-child relationship, mother's love, mother-son relationship, mother-daughter, mother-son, divorced mother, widowed mother, divorced men, [divorce, divorced], [Divorcees, divorcee], [single mother, single mothers], first wife, ex-wife, presidential wife, missing wife, twelfth wife, deceased wife, questioning parents, parental questioning, neglectful father, ailing father, absent father, missing father, abusive father, Ex-wives, [ex-husband, ex-husbands], billionaire husband, Famous husband, parent-child bonds, parent-child relationship, lifelong bonds, parent-child, [father-child relationship, father-child relationships], divorce consequences, divorce aftermath, single mom, single dad, single motherhood, stay-at-home mom, [Single parents, single parent, single parenting], Manipulative parents, [Overbearing parent, Overbearing parents], Daughter's Condition, daughter's life, Daughter's struggle, daughter's story, Daughter's expectations, parental dysfunction, Determined father, Troubled father, Targeted father, desperate father, daughter killed, parental love, Fatherly love, Motherly love, Parental care, maternal love, [parenting, parents], mother's milk, mother's life, mother's, [mother, mothers], Motherly protection, Mothers' support, Motherly role, Maternal instinct, Unmarried mothers, Solo Mothers, ailing mother, dissolute father, dying father, late father, protective father, difficult father, alcoholic father, father's archive, [father, fathers], Abducted wife, Mom, grieving neighbor, Grieving widow, grieving mother, grieving father, missing daughter, missing mother, missing mom, mother's support, Mothers and daughters, mother's influence, Mother's insistence, parental anguish, parental grief, Parental struggles, parental dilemma, parental issues, desperate parents, ...]"
2,344,"[ski-movie, ski film, Cinematic sketches, Cinematic essay, Filmmaker's aesthetics, Filmmaker persona, Filmmaker's project, filmmaker's perspective, Filmmaker fragments, [Experimental filmmaking, experimental filmmaker, experimental

Gamma value: 2.0


,themeCount,themes
0,282,"[ski-movie, ski film, Cinematic sketches, Cinematic essay, Filmmaker's aesthetics, Filmmaker persona, Filmmaker's project, filmmaker's perspective, Filmmaker fragments, [Experimental filmmaking, experimental filmmaker, experimental filmmakers], stag films, art film, handcrafted film, artist film, artistic cinema, concert film, artistic film circles, unfinished film, paint on film, touching film, [Experimental films, experimental film], Action star, Movie man, movies within movies, movie props, handmade, handcrafted filmmaking, anime film, Anime world, anime-inspired, action film, screenplay, screenplay scenes, roleplay, [film, filming, films], [show, shows], Italian Filmmaker, Audacious filmmaker, Bipolar filmmaker, Dedicated filmmaker, filmmaker relationship, filmmaker couple, Filmmaker's Notebook, Movie set, Movie project, On set, movie production, cinematic history, Cinematographic origin, cinematic passion, cinematic journey, cinema history, cinematic world, cinematic, cinematic universe, cinematic knowledge, cinematic image, cinematic future, film grain, film frames, Cannes festival, Cannes Film Festival, [Film festival, film festivals], Travelling cinema, Expanded cinema, aspiring screenwriter, Screenwriting professor, filmmaking aspirations, aspiring filmmaker, Film actor, Film hero, popular movie, movie about movie, movie lovers, Film archive, Film collector, film artifacts, film institution, Cinema closures, cinema's future, cinema outing, cinema business, [Cinemas, cinema], Cinema evolution, Cinema pioneer, Cinema Odyssey, Movie premiere, Film promotion, Premiere day, movie reception, premiere, Personal film, Home film, Role search, Roles, [role-play, role-playing], [student film, student films], flickering visuals, flicker film, flickering tracings, ...]"
1,241,"[Extraterrestrial communication, extraterrestrial beings, Extraterrestrial evidence, Extraterrestrial phenomenon, [extraterrestrial, extraterrestrials], Extra-terrestrial force, extraterrestrial invasion, Extraterrestrial life, mysterious object, Mysterious alien, mysterious body, mysterious entity, mysterious ship, stray cats, lost cat, indoor cats, [stray dog, stray dogs], Alien invention, alien technology, Alien civilization, [Alien invasions, alien invasion], [Alien presence, Alien present], Moon civilization, colorless world, Colourless, wild creatures, native animals, hidden creatures, saving creature, creatures flee, invisible creatures, forest creatures, wildlife, [creature, creatures], Barren planet, Distant planet, Barren path, Barren land, fauna, control freak, freak, forest monster, [swamp monster, swamp monsters], invasion, infestation, [Invader, Invaders], degenerate creatures, mutant creature, vile monsters, mysterious force, mysterious organization, mysterious motive, Frozen planet, alien planet, Stolen planet, dream planet, Missing cat, Starving cat, Talking cat, Cat's death, Cat son, wildlife crime, wildlife survival, Wildlife codes, wildlife photography, Parasitic aliens, Alien lifeforms, Space aliens, alien beings, Alien visitors, homicidal aliens, Mistaken intentions, mistaken target, mistaken, [Mistaken identities, mistaken identity], Undead monster, feline devotion, cat companion, feline companion, feline, mystery creature, [bizarre creature, bizarre creatures], [mysterious creature, mysterious creatures], [Strange creature, strange creatures], alien environment, alien encounter, [alien, alienating, alienation, aliens], flesh creatures, Dry Land, Dry hole, Bizarre fish, extraterrestrial craft, space invaders, Alien shapeshifter, Alien Life Forms, Ancient extraterrestrials, astral creature, miscommunication, animatronic horrors, animated monsters, ...]"
2,233,"[spiritual significance, spiritual imagery, spiritual connections, [Spiritualism, spiritual, spirituality], [Spiritual community, spiritual communication], dangerous path, [Dangerous adventures, dangerous adventure], spiritual longing, Spiritual presence,

Gamma value: 4.0


...

,themeCount,themes
0,27,"[human condition, human intelligence, [Humanism, Humanities, human, humanity, humanization, humans], Human exceptionalism, human perception, human vision, human spirit, [Humanist, Humanistic], human face, human nature, Humanly, human curiosity, shared humanity, human existence, human race, human world, Human instincts, [Human emotion, human emotions], human creativity, Humanity's rebirth, inhuman, good human, post-human, human motion, Human lives, Humano, hope for humanity]"
1,26,"[Bread and Blood, Blood and tears, Blood conveyor, Blood trade, bloodshed, bloodlust, Blood shed, blood lust, blood ritual, Blood stains, Bleeds, blood pact, blood money, blood dreams, Lust for blood, blood path, blood, blood supply, blood room, Blood Red, bloodbath, Blood cult, bloodlines, blood-soaked nights, blood addiction, Accursed blood]"
2,25,"[Bureaucrats, [Civil servant, civil servants], riot police, IPS officer, police inspector, police chief, Indian police, police constable, Police captain, [police officer, police officers], law enforcement, Law enforcement accountability, [police, policing], policewoman, policeman, time police, Senior constable, [Constable, constables], Patrolman, Policeman neighbor, missing officer, service officer, Met Police, police force, [Police state, police station]]"
3,25,"[romantic square, interracial romance, [romance, romances], wartime romance, romantic comedy, Romantic satire, Romantic thriller, romantic, rom-com, romantic action, romance novel, maritime romance, novelist, author's novel, novel, romance novelist, Steamy romance, Romantic love, bromance, [Romance scams, romance scam], unfinished novel, sailor romance, romantic language, romance king, romance writer]"
4,25,"[extraordinary life, ordinary lives, ordinary life, routine life, normal life, little life, simple life, modest life, structured life, repetitive life, traditional life, easy life, Life structure, Constructed life, Whole life, Stable Life, ordinary people, ordinary scenes, simplicity, hard life, Retirement life, PA life, public life, short life, Normalcy]"


Gamma value: 512.0


,themeCount,themes
0,20,"[Community chaos, police chaos, chaos, Collective disarray, Mass chaos, chaotic events, mayhem, havoc, home-brewed chaos, chaotic, order in chaos, chaotic world, chaos theory, Hilarious chaos, labyrinthine chaos, Chaotic poetry, chaotic lifestyle, chaotic response, erratic, Disrupted lifestyle]"
1,18,"[Sanctuary City, Sacred city, Sacred Path, Sacred place, sacred nature, sacred site, Sacred lake, pilgrimage site, sacred images, sacred, Sacred objects, sacred territory, indigenous territory, Holy Place, holy ground, meaningful images, special places, different places]"
2,18,"[child escape, Train escape, escape attempt, no escape, school escape, [Escapes, escape, escapism], clinic escape, escape routine, [escape plan, escape plans], Escaping war, mental escape, Great escape, mind escape, no way out, escape rooms, Escape pod, Trapped room, escapee]"
3,17,"[Corporate competition, Corporate politics, Complex competition, [Tournaments, tournament], Stone bridge, [bridge, bridges], Cutthroat competition, Intense competition, competitive pursuit, [Competitions, competition, competitive], talent competition, ring competition, bridge competition, team competition, Academic competition, Talentshow, school talent show]"
4,17,"[dance routines, dance performances, dance theater, dance vignettes, [dance, dances, dancing], [dancer, dancers], time's dance, cultural dances, Collective dance, [Drag performers, drag performer], dancing friends, Bailando, dance contests, vogue dancing, school dance, belly dance, Tanz]"


Gamma value: 1024.0


,themeCount,themes
0,15,"[Seaside village, beach house, Beachside getaway, beach setting, Paradise beach, Beach bar, beachbreak, The Beach, [beach, beaches], Beach romance, Beach reunion, Beach Punk, beach closure, beach vendors, The Beach Boys]"
1,14,"[River murder, river tragedy, River life, riverside, River Boy, riverine, River setting, River guides, [Flowing river, flowing rivers], [river, rivers], River Cam, waterways, Rivulet, [Flow, flowing]]"
2,13,"[Amateur boxing, Amateur tournament, [Tournaments, tournament], Stone bridge, [bridge, bridges], Cutthroat competition, Intense competition, competitive pursuit, [Competitions, competition, competitive], ring competition, bridge competition, team competition, Academic competition]"
3,13,"[Subjugated humanity, Suffering humanity, suffering, Physical suffering, patient suffering, Souffrir, Sufrimiento, Psychological suffering, physical pain, Physical ailment, silent pain, [pain, painful], Human anguish]"
4,13,"[feminist mayor, feminista, feminist initiative, feminist collective, Feminist media, Feminist filmmakers, Eco-feminist, feminist protest, feminist reimagining, [Feminisms, feminism], French feminist, Feminist themes, feminist sci-fi]"


## For individual themes, see what other themes drop out of their communities as you increase gamma

In [438]:
def get_proprety_df(property_name):
    property_df = gds.graph.nodeProperties.stream(g_themes2,
                                                  [property_name],
                                                  db_node_properties = ["description", "descriptions"],
                                                  separate_property_columns=True)
    property_df.loc[pd.notna(property_df['descriptions']), 'descriptions'] = property_df.loc[pd.notna(property_df['descriptions']), 'descriptions'].map(lambda x: ", ".join(x))
    property_df['description'] = property_df.apply(lambda row: row['description'] if row['description'] else row['descriptions'], axis=1)
    property_df.drop('descriptions', axis=1, inplace=True)
    return property_df

property_dfs = {property_name: get_proprety_df(property_name)
                for property_name in [f"leidenGamma{gamma}" for gamma in [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0]]}

In [439]:
def get_cluster_for_theme(property_name, theme):
    theme_df = property_dfs[property_name]
    result_list = [theme]
    if theme in theme_df['description'].to_list():
        theme_cluster_id = theme_df.loc[theme_df['description']==theme, property_name].iloc[0]
        result_list = theme_df.loc[theme_df[property_name]==theme_cluster_id, "description"].to_list()
    return result_list

In [440]:
def compare_gamma_for_theme(theme, gammas):
    column_names = [f"leidenGamma{gamma}" for gamma in gammas]
    lists = [get_cluster_for_theme(column, theme) for column in column_names]
    output_df = pd.DataFrame()
    output_df.loc[column_names[0], "final_themes"] = ", ".join(lists[0])
    for i in range(1, len(gammas)):
        output_df.loc[f"{column_names[i-1]} to {column_names[i]}", "dropped_themes"] = ", ".join([theme for theme in lists[i-1] if theme not in lists[i]])
        output_df.loc[f"{column_names[i-1]} to {column_names[i]}", "added_themes"] = ", ".join([theme for theme in lists[i] if theme not in lists[i-1]])
        output_df.loc[f"{column_names[i-1]} to {column_names[i]}", "final_themes"] = ", ".join(lists[i])
    return output_df

In [441]:
compare_gamma_for_theme("christmas", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Christmas setting, Christmas Jam, Christmas World, Christmas magic, Christmas wedding, White Christmas, Christmas Variety, Christmas holidays, Christmas break, christmas, Christmas decorations, Navidad, Christmas celebration, Christmas celebrations, Christmas miracle, Christmas nightmare, Christmas curse, Christmas spirit, Christmas mystery, Christmas wish, Christmas wishes, Holiday traditions, Christmas memories, Christmas tradition, Christmas traditions, Christmas sweaters, Christmas lights, Christmas movie, Christmas eve, Christmas party, Christmas story, spicy Christmas, Xmas, Christmas ambience, Christmas terror, Christmas love, Christmas tragedy, Christmas proposal, Christmas gift, Christmas reunion, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas ornaments, Christmas tree, Christmas trees, Christmas train, Christmas songs, Christmas Eve wedding, Carol of the Bells, Christmas carols, Caroling, carols",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Christmas setting, Christmas Jam, Christmas World, Christmas magic, Christmas wedding, White Christmas, Christmas Variety, Christmas holidays, Christmas break, christmas, Navidad, Christmas celebration, Christmas celebrations, Christmas miracle, Christmas nightmare, Christmas curse, Christmas spirit, Christmas mystery, Christmas wish, Christmas wishes, Christmas sweaters, Christmas movie, Christmas eve, Christmas party, Christmas story, spicy Christmas, Xmas, Christmas ambience, Christmas terror, Christmas love, Christmas tragedy, Christmas proposal, Christmas gift, Christmas reunion, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas Eve wedding","Christmas decorations, Holiday traditions, Christmas memories, Christmas tradition, Christmas traditions, Christmas lights, Christmas ornaments, Christmas tree, Christmas trees, Christmas train, Christmas songs, Carol of the Bells, Christmas carols, Caroling, carols",
leidenGamma64.0 to leidenGamma128.0,"Christmas holidays, Christmas break, christmas, Navidad, Xmas","Christmas setting, Christmas Jam, Christmas World, Christmas magic, Christmas wedding, White Christmas, Christmas Variety, Christmas celebration, Christmas celebrations, Christmas miracle, Christmas nightmare, Christmas curse, Christmas spirit, Christmas mystery, Christmas wish, Christmas wishes, Christmas sweaters, Christmas movie, Christmas eve, Christmas party, Christmas story, spicy Christmas, Christmas ambience, Christmas terror, Christmas love, Christmas tragedy, Christmas proposal, Christmas gift, Christmas reunion, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas Eve wedding",
leidenGamma128.0 to leidenGamma256.0,"Christmas magic, christmas, Xmas","Christmas holidays, Christmas break, Navidad",Christmas magic
leidenGamma256.0 to leidenGamma512.0,"Christmas holidays, Christmas break, christmas, Navidad, Xmas",Christmas magic,"Christmas holidays, Christmas break, Navidad"
leidenGamma512.0 to leidenGamma1024.0,"christmas, Navidad, Xmas","Christmas holidays, Christmas break",


In [460]:
compare_gamma_for_theme("Christmas eve", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Christmas setting, Christmas Jam, Christmas World, Christmas magic, Christmas wedding, White Christmas, Christmas Variety, Christmas holidays, Christmas break, christmas, Christmas decorations, Navidad, Christmas celebration, Christmas celebrations, Christmas miracle, Christmas nightmare, Christmas curse, Christmas spirit, Christmas mystery, Christmas wish, Christmas wishes, Holiday traditions, Christmas memories, Christmas tradition, Christmas traditions, Christmas sweaters, Christmas lights, Christmas movie, Christmas eve, Christmas party, Christmas story, spicy Christmas, Xmas, Christmas ambience, Christmas terror, Christmas love, Christmas tragedy, Christmas proposal, Christmas gift, Christmas reunion, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas ornaments, Christmas tree, Christmas trees, Christmas train, Christmas songs, Christmas Eve wedding, Carol of the Bells, Christmas carols, Caroling, carols",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Christmas setting, Christmas Jam, Christmas World, Christmas magic, Christmas wedding, White Christmas, Christmas Variety, Christmas holidays, Christmas break, christmas, Navidad, Christmas celebration, Christmas celebrations, Christmas miracle, Christmas nightmare, Christmas curse, Christmas spirit, Christmas mystery, Christmas wish, Christmas wishes, Christmas sweaters, Christmas movie, Christmas eve, Christmas party, Christmas story, spicy Christmas, Xmas, Christmas ambience, Christmas terror, Christmas love, Christmas tragedy, Christmas proposal, Christmas gift, Christmas reunion, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas Eve wedding","Christmas decorations, Holiday traditions, Christmas memories, Christmas tradition, Christmas traditions, Christmas lights, Christmas ornaments, Christmas tree, Christmas trees, Christmas train, Christmas songs, Carol of the Bells, Christmas carols, Caroling, carols",
leidenGamma64.0 to leidenGamma128.0,"Christmas setting, Christmas World, Christmas wedding, White Christmas, Christmas celebration, Christmas celebrations, Christmas movie, Christmas eve, Christmas party, Christmas reunion, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas Eve wedding","Christmas Jam, Christmas magic, Christmas Variety, Christmas holidays, Christmas break, christmas, Navidad, Christmas miracle, Christmas nightmare, Christmas curse, Christmas spirit, Christmas mystery, Christmas wish, Christmas wishes, Christmas sweaters, Christmas story, spicy Christmas, Xmas, Christmas ambience, Christmas terror, Christmas love, Christmas tragedy, Christmas proposal, Christmas gift",
leidenGamma128.0 to leidenGamma256.0,"Christmas wedding, Christmas eve, Christmas party, Family Christmas, Christmas competition, Christmas market, Santa competition, Christmas Eve wedding","Christmas setting, Christmas World, White Christmas, Christmas celebration, Christmas celebrations, Christmas movie, Christmas reunion",
leidenGamma256.0 to leidenGamma512.0,"Christmas eve, Christmas Eve wedding","Christmas wedding, Christmas party, Family Christmas, Christmas competition, Christmas market, Santa competition",
leidenGamma512.0 to leidenGamma1024.0,"Christmas eve, Christmas Eve wedding",,


In [442]:
compare_gamma_for_theme("Beatles", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Beatles, Paul McCartney",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Beatles, Paul McCartney",,
leidenGamma64.0 to leidenGamma128.0,"Beatles, Paul McCartney",,
leidenGamma128.0 to leidenGamma256.0,"Beatles, Paul McCartney",,
leidenGamma256.0 to leidenGamma512.0,"Beatles, Paul McCartney",,
leidenGamma512.0 to leidenGamma1024.0,"Beatles, Paul McCartney",,


In [443]:
compare_gamma_for_theme("soldier", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,soldier,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,soldier,,
leidenGamma64.0 to leidenGamma128.0,soldier,,
leidenGamma128.0 to leidenGamma256.0,soldier,,
leidenGamma256.0 to leidenGamma512.0,soldier,,
leidenGamma512.0 to leidenGamma1024.0,soldier,,


In [444]:
compare_gamma_for_theme("Family bond", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,Family bond,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,Family bond,,
leidenGamma64.0 to leidenGamma128.0,Family bond,,
leidenGamma128.0 to leidenGamma256.0,Family bond,,
leidenGamma256.0 to leidenGamma512.0,Family bond,,
leidenGamma512.0 to leidenGamma1024.0,Family bond,,


In [445]:
compare_gamma_for_theme("dark place", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Darkness to light, Dark Power, dark journey, darkness vs. light, Blue thoughts, dark cracks, dark characters, Dark Void, darkest times, lights out, lost light, black void, Darkest Hour, Dark Fantasy, dark fantasies, dust, Dustlight, Dark Light, dark themes, dark self, Dark love, Dark interest, Dark realm, Dark legacy, Dark companion, dark child, inner darkness, dark place, out of darkness, Darkness beneath, dark world, Darkness and silence, Darkness within, dark abyss, divine darkness, dark, darkness, dark energy, dark presence, dark magic, dark force, dark forces, Deathful power, dark house, Dark Cost, dark vision, Dark Haven, dark manipulation, dark twist, Dark Figure, dark web, Dark ritual, dark network, Dark tale, Dark Prison, dark path, sinister world, Dark Highway, Dark forests, dark forest, dark road, dark thoughts, Deep dark, dark side, dark turn, dark time, dark corridors, dark city, dark room, dark office, dark stranger, dark holes, dark tunnel, dark walls, dark plot, dark scheme, dark cloud, dark clouds",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Darkness to light, dark journey, darkness vs. light, Blue thoughts, dark cracks, Dark Void, darkest times, black void, Darkest Hour, Dark Fantasy, dark fantasies, dark themes, Dark realm, inner darkness, dark place, out of darkness, Darkness beneath, dark world, Darkness and silence, Darkness within, dark abyss, divine darkness, dark, darkness, dark house, Dark Haven, dark twist, dark web, Dark ritual, dark network, Dark tale, Dark Prison, dark path, sinister world, Dark Highway, Dark forests, dark forest, dark road, dark thoughts, Deep dark, dark turn, dark time, dark corridors, dark city, dark room, dark office, dark holes, dark tunnel, dark walls, dark scheme, dark cloud, dark clouds","Dark Power, dark characters, lights out, lost light, dust, Dustlight, Dark Light, dark self, Dark love, Dark interest, Dark legacy, Dark companion, dark child, dark energy, dark presence, dark magic, dark force, dark forces, Deathful power, Dark Cost, dark vision, dark manipulation, Dark Figure, dark side, dark stranger, dark plot",
leidenGamma64.0 to leidenGamma128.0,"dark journey, darkness vs. light, Blue thoughts, dark cracks, darkest times, Darkest Hour, Dark Fantasy, dark fantasies, dark themes, Dark realm, dark place, dark world, dark, darkness, dark house, Dark Haven, dark twist, dark web, Dark ritual, dark network, Dark tale, dark path, sinister world, Dark Highway, Dark forests, dark forest, dark road, dark thoughts, Deep dark, dark turn, dark time, dark corridors, dark city, dark room, dark office, dark holes, dark tunnel, dark walls, dark scheme, dark cloud, dark clouds","Darkness to light, Dark Void, black void, inner darkness, out of darkness, Darkness beneath, Darkness and silence, Darkness within, dark abyss, divine darkness, Dark Prison",
leidenGamma128.0 to leidenGamma256.0,"dark themes, dark place, dark house, dark twist, dark path, dark thoughts, dark turn, dark corridors, dark room, dark office, dark holes, dark tunnel, dark walls, dark scheme, dark journey, Blue thoughts, dark cracks","darkness vs. light, darkest times, Darkest Hour, Dark Fantasy, dark fantasies, Dark realm, dark world, dark, darkness, Dark Haven, dark web, Dark ritual, dark network, Dark tale, sinister world, Dark Highway, Dark forests, dark forest, dark road, Deep dark, dark time, dark city, dark cloud, dark clouds",
leidenGamma256.0 to leidenGamma512.0,"dark place, dark house, dark corridors, dark room, dark office, dark holes, dark walls, dark cracks","dark themes, dark twist, dark path, dark thoughts, dark turn, dark tunnel, dark scheme, dark journey, Blue thoughts",
leidenGamma512.0 to leidenGamma1024.0,"dark place, dark house, dark office, dark holes","dark corridors, dark room, dark walls, dark cracks",


In [446]:
compare_gamma_for_theme("horror comedy", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Horror-Noir, survival horror, horror convention, Creepshow, Creeper, Zombie bite, gothic horror, horror genre, fantastical horror, Found footage horror, horror realm, noir, Human decomposition, Horror games, horror videos, Horror homage, horror stories, horror story, teen horror, J-horror, family horror, Real horror, zombie horror, silent horror, body horror, Turkish horror, horror footage, family terror, B-horror, Holiday Horrors, Holiday horror, horror, horrors, Horror filming, horror film, horror films, Horror movies, horror movie, Horror reboot, Horror series, horror franchise, Horror classic, horror classics, psycho-horror, horror-comedy, horror compilation, horror comedy, Horror icons, Horror elements, Horror villains, Horror legend, horror legends, Bodily horror, Creeping horror, Basement horror, Horror setting, horror anthology",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Horror-Noir, horror convention, Creepshow, Creeper, Zombie bite, gothic horror, horror genre, fantastical horror, Found footage horror, horror realm, noir, Human decomposition, Horror games, horror videos, Horror homage, horror stories, horror story, teen horror, J-horror, family horror, Real horror, zombie horror, silent horror, body horror, Turkish horror, horror footage, family terror, B-horror, Holiday Horrors, Holiday horror, horror, horrors, Horror filming, horror film, horror films, Horror movies, horror movie, Horror reboot, Horror series, horror franchise, Horror classic, horror classics, psycho-horror, horror-comedy, horror compilation, horror comedy, Horror icons, Horror elements, Horror villains, Horror legend, horror legends, Bodily horror, Creeping horror, Basement horror, Horror setting, horror anthology",survival horror,
leidenGamma64.0 to leidenGamma128.0,"Horror-Noir, gothic horror, fantastical horror, noir, J-horror, B-horror, psycho-horror, horror-comedy, horror compilation, horror comedy, horror anthology","horror convention, Creepshow, Creeper, Zombie bite, horror genre, Found footage horror, horror realm, Human decomposition, Horror games, horror videos, Horror homage, horror stories, horror story, teen horror, family horror, Real horror, zombie horror, silent horror, body horror, Turkish horror, horror footage, family terror, Holiday Horrors, Holiday horror, horror, horrors, Horror filming, horror film, horror films, Horror movies, horror movie, Horror reboot, Horror series, horror franchise, Horror classic, horror classics, Horror icons, Horror elements, Horror villains, Horror legend, horror legends, Bodily horror, Creeping horror, Basement horror, Horror setting",
leidenGamma128.0 to leidenGamma256.0,"horror-comedy, horror compilation, horror comedy, horror anthology","Horror-Noir, gothic horror, fantastical horror, noir, J-horror, B-horror, psycho-horror",
leidenGamma256.0 to leidenGamma512.0,"horror-comedy, horror compilation, horror comedy, horror anthology",,
leidenGamma512.0 to leidenGamma1024.0,"horror-comedy, horror comedy","horror compilation, horror anthology",


In [447]:
compare_gamma_for_theme("LGBTQ+", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"LGBT refugees, LGBTQ+ Iran, 2SLGBTQ+, LGBTQ pride, LGBTQI+ families, LGBTQIAP+, LGBTQ2IA+, LGBTQ+ pageant, Beauty pageants, beauty pageant, LGBTQ history, LGBTQ+, LGBT, LGBT youth, LGBTQ, LGBTQIA+, LGBT community, LGBTQ identity, LGBTQ+ identity, LGBTQ visibility, LGBTQ+ community, LGBTQ community, LGBTQIA+ community, LGBTQI+ community, LGBT+ acceptance, LGBTQ inclusion, LGBTQ+ archive, LGBT activism, queer activism, LGBTQIA+ activism, LGBTQ activism, LGBTQ discrimination, LGBTIQ+ migration",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"LGBTQ+ Iran, 2SLGBTQ+, LGBTQ pride, LGBTQI+ families, LGBTQIAP+, LGBTQ2IA+, LGBTQ+ pageant, Beauty pageants, beauty pageant, LGBTQ history, LGBTQ+, LGBT, LGBT youth, LGBTQ, LGBTQIA+, LGBT community, LGBTQ identity, LGBTQ+ identity, LGBTQ visibility, LGBTQ+ community, LGBTQ community, LGBTQIA+ community, LGBTQI+ community, LGBT+ acceptance, LGBTQ inclusion, LGBTQ+ archive, LGBTQ discrimination","LGBT refugees, LGBT activism, queer activism, LGBTQIA+ activism, LGBTQ activism, LGBTIQ+ migration",
leidenGamma64.0 to leidenGamma128.0,"LGBTQ+ Iran, 2SLGBTQ+, LGBTQ pride, LGBTQIAP+, LGBTQ2IA+, LGBTQ+ pageant, Beauty pageants, beauty pageant, LGBTQ history, LGBTQ+, LGBT, LGBT youth, LGBTQ, LGBTQIA+, LGBTQ identity, LGBTQ+ identity, LGBTQ visibility, LGBTQ inclusion, LGBTQ+ archive, LGBTQ discrimination","LGBTQI+ families, LGBT community, LGBTQ+ community, LGBTQ community, LGBTQIA+ community, LGBTQI+ community, LGBT+ acceptance",
leidenGamma128.0 to leidenGamma256.0,"LGBTQ+, LGBTQ+ archive, LGBTQ+ Iran","2SLGBTQ+, LGBTQ pride, LGBTQIAP+, LGBTQ2IA+, LGBTQ+ pageant, Beauty pageants, beauty pageant, LGBTQ history, LGBT, LGBT youth, LGBTQ, LGBTQIA+, LGBTQ identity, LGBTQ+ identity, LGBTQ visibility, LGBTQ inclusion, LGBTQ discrimination",
leidenGamma256.0 to leidenGamma512.0,"LGBTQ+, LGBTQ+ Iran",LGBTQ+ archive,
leidenGamma512.0 to leidenGamma1024.0,"LGBTQ+ Iran, LGBTQ+",,


In [451]:
compare_gamma_for_theme("friendship, friendships", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"friendship adventure, friendship strain, Twisted friendship, misfit friendship, unlikely friendship, toxic friendship, strained friendship, unusual friendship, New friendships, unexpected friendship, Unlikely companionship, friendship test, friendship tested, enduring friendship, friend, friends, friendship unravels, friendships tested, friendship cracks, Friendly, female friendship, male friendship, Trans friendship, strong friendship, girl friendships, friendship, friendships, platonic friendship, friendship advice, Friendship dynamics, Friendship loyalty, friendship torn, Friendship bonds, friendship bond",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"friendship adventure, friendship strain, Twisted friendship, misfit friendship, unlikely friendship, toxic friendship, strained friendship, unusual friendship, New friendships, unexpected friendship, Unlikely companionship, friendship test, friendship tested, enduring friendship, friendship unravels, friendships tested, friendship cracks, female friendship, male friendship, Trans friendship, strong friendship, girl friendships, friendship, friendships, platonic friendship, friendship advice, Friendship dynamics, Friendship loyalty, friendship torn, Friendship bonds, friendship bond","friend, friends, Friendly",
leidenGamma64.0 to leidenGamma128.0,"friendship adventure, friendship strain, New friendships, friendship test, friendship tested, enduring friendship, friendship unravels, friendships tested, friendship cracks, female friendship, male friendship, Trans friendship, strong friendship, girl friendships, friendship, friendships, platonic friendship, friendship advice, Friendship dynamics, Friendship loyalty, Friendship bonds, friendship bond","Twisted friendship, misfit friendship, unlikely friendship, toxic friendship, strained friendship, unusual friendship, unexpected friendship, Unlikely companionship, friendship torn",
leidenGamma128.0 to leidenGamma256.0,"female friendship, male friendship, Trans friendship, strong friendship, girl friendships, friendship, friendships, platonic friendship, New friendships, enduring friendship","friendship adventure, friendship strain, friendship test, friendship tested, friendship unravels, friendships tested, friendship cracks, friendship advice, Friendship dynamics, Friendship loyalty, Friendship bonds, friendship bond",
leidenGamma256.0 to leidenGamma512.0,"friendship, friendships, friendship advice, Friendship dynamics, Friendship loyalty, Friendship bonds, friendship bond, friendship adventure, friendship strain, New friendships","female friendship, male friendship, Trans friendship, strong friendship, girl friendships, platonic friendship, enduring friendship","friendship advice, Friendship dynamics, Friendship loyalty, Friendship bonds, friendship bond, friendship adventure, friendship strain"
leidenGamma512.0 to leidenGamma1024.0,"New friendships, friendship, friendships","friendship advice, Friendship dynamics, Friendship loyalty, Friendship bonds, friendship bond, friendship adventure, friendship strain",


In [450]:
compare_gamma_for_theme("friend, friends", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"friendship adventure, friendship strain, Twisted friendship, misfit friendship, unlikely friendship, toxic friendship, strained friendship, unusual friendship, New friendships, unexpected friendship, Unlikely companionship, friendship test, friendship tested, enduring friendship, friend, friends, friendship unravels, friendships tested, friendship cracks, Friendly, female friendship, male friendship, Trans friendship, strong friendship, girl friendships, friendship, friendships, platonic friendship, friendship advice, Friendship dynamics, Friendship loyalty, friendship torn, Friendship bonds, friendship bond",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"dearest friend, best friend, best friends, lifelong friends, close friends, rich friends, ex-best friend, friend, friends, prestigious friends, Friendly, school friends, loyal friend, faithful friend, cherished friend, true friend","friendship adventure, friendship strain, Twisted friendship, misfit friendship, unlikely friendship, toxic friendship, strained friendship, unusual friendship, New friendships, unexpected friendship, Unlikely companionship, friendship test, friendship tested, enduring friendship, friendship unravels, friendships tested, friendship cracks, female friendship, male friendship, Trans friendship, strong friendship, girl friendships, friendship, friendships, platonic friendship, friendship advice, Friendship dynamics, Friendship loyalty, friendship torn, Friendship bonds, friendship bond","dearest friend, best friend, best friends, lifelong friends, close friends, rich friends, ex-best friend, prestigious friends, school friends, loyal friend, faithful friend, cherished friend, true friend"
leidenGamma64.0 to leidenGamma128.0,"dearest friend, best friend, best friends, lifelong friends, close friends, rich friends, ex-best friend, friend, friends, prestigious friends, Friendly, school friends, loyal friend, faithful friend, cherished friend, true friend",,
leidenGamma128.0 to leidenGamma256.0,"school friends, loyal friend, faithful friend, cherished friend, true friend, dearest friend, best friend, best friends, lifelong friends, close friends, rich friends, ex-best friend, friend, friends, prestigious friends, Friendly",,
leidenGamma256.0 to leidenGamma512.0,"school friends, best friend, best friends, lifelong friends, close friends, rich friends, ex-best friend, friend, friends, prestigious friends, Friendly","loyal friend, faithful friend, cherished friend, true friend, dearest friend",
leidenGamma512.0 to leidenGamma1024.0,"friend, friends, Friendly","school friends, best friend, best friends, lifelong friends, close friends, rich friends, ex-best friend, prestigious friends",


In [155]:
compare_gamma_for_theme("high school friends", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"Acquaintance, acquaintances, old acquaintances, best friend, best friends, faithful friend, true friend, ex-best friend, school friends, close friends, enemy, old enemy, enemy positions, Frenemy, Friends turned enemies, friend or foe, frenemies, friends-turned-foes, friend, Friendly, friends, friend group, high school friends, old enemies, old friend, old friends, rich friends, New friendships, reunited friends, Teenage friends",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Teenage friends, Acquaintance, acquaintances, old acquaintances, best friend, best friends, faithful friend, true friend, ex-best friend, school friends, close friends, enemy, old enemy, enemy positions, Frenemy, Friends turned enemies, friend or foe, frenemies, friends-turned-foes, friend, Friendly, friends, friend group, high school friends, old enemies, old friend, old friends, rich friends, New friendships, reunited friends",,
leidenGamma64.0 to leidenGamma128.0,"Acquaintance, acquaintances, old acquaintances, best friend, best friends, faithful friend, true friend, ex-best friend, school friends, close friends, enemy, old enemy, enemy positions, Frenemy, Friends turned enemies, friend or foe, frenemies, friends-turned-foes, friend, Friendly, friends, friend group, high school friends, old enemies, old friend, old friends, rich friends, New friendships, reunited friends, Teenage friends",,
leidenGamma128.0 to leidenGamma256.0,"school friends, close friends, friend, Friendly, friends, friend group, high school friends, old friends, rich friends, New friendships, reunited friends, Teenage friends","Acquaintance, acquaintances, old acquaintances, best friend, best friends, faithful friend, true friend, ex-best friend, enemy, old enemy, enemy positions, Frenemy, Friends turned enemies, friend or foe, frenemies, friends-turned-foes, old enemies, old friend",
leidenGamma256.0 to leidenGamma512.0,"school friends, close friends, friend group, high school friends, old friends, rich friends, New friendships, reunited friends, siblings reunited, Teenage friends","friend, Friendly, friends",siblings reunited
leidenGamma512.0 to leidenGamma1024.0,"high school friends, reunited friends, siblings, siblings reunited","school friends, close friends, friend group, old friends, rich friends, New friendships, Teenage friends",siblings


In [452]:
compare_gamma_for_theme("old woman", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"old soldier, Care home, old maid, Older man, old widower, older men, Old Men, middle-aged man, Retired couple, Wise elder, Elder, Elders, Dorothy, young man, elders' passing, Maids, maid, home birth, Sixty-year-old women, elderly women, home care, elderly, old couple, old man, old woman, old lady, old guys, elderly woman, elderly gentleman, Elderly spinster, elderly aunt, elderly Dorothy, senior woman, Elderly patient, Elderly widow, elderly couple, elderly wrestler, elderly love, Elderly witch, elderly sisters, elderly man, elderly lady, elderly maid, elderly mother, elderly student, bedridden woman, older woman, elderly protagonist, Senior citizen, senior citizens, Elderly father, elderly disguise, Village elders, village elder, old age, senility, elderly life, ageing, aging, Elderly coach, Elderly daughter, elderly rebellion, elderly home, Elderly uncle, Elderly farmer, Elderly priest, Elderly guardian, elderly parent, old age home, elderly care, real elder, aging parents, Senior, seniors, Elderly companionship, Aged care, nursing home, nursing homes, retirement home, retirement homes",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"old soldier, old maid, Older man, old widower, older men, Old Men, old couple, old man, old woman, old lady, old guys, senior woman, older woman","Care home, middle-aged man, Retired couple, Wise elder, Elder, Elders, Dorothy, young man, elders' passing, Maids, maid, home birth, Sixty-year-old women, elderly women, home care, elderly, elderly woman, elderly gentleman, Elderly spinster, elderly aunt, elderly Dorothy, Elderly patient, Elderly widow, elderly couple, elderly wrestler, elderly love, Elderly witch, elderly sisters, elderly man, elderly lady, elderly maid, elderly mother, elderly student, bedridden woman, elderly protagonist, Senior citizen, senior citizens, Elderly father, elderly disguise, Village elders, village elder, old age, senility, elderly life, ageing, aging, Elderly coach, Elderly daughter, elderly rebellion, elderly home, Elderly uncle, Elderly farmer, Elderly priest, Elderly guardian, elderly parent, old age home, elderly care, real elder, aging parents, Senior, seniors, Elderly companionship, Aged care, nursing home, nursing homes, retirement home, retirement homes",
leidenGamma64.0 to leidenGamma128.0,"old maid, Dorothy, Maids, maid, Sixty-year-old women, elderly women, old couple, old woman, old lady, elderly woman, elderly Dorothy, senior woman, elderly sisters, elderly lady, elderly maid, elderly student, bedridden woman, older woman","old soldier, Older man, old widower, older men, Old Men, old man, old guys","Dorothy, Maids, maid, Sixty-year-old women, elderly women, elderly woman, elderly Dorothy, elderly sisters, elderly lady, elderly maid, elderly student, bedridden woman"
leidenGamma128.0 to leidenGamma256.0,"old couple, old woman, old lady, senior woman, older woman, old maid","Dorothy, Maids, maid, Sixty-year-old women, elderly women, elderly woman, elderly Dorothy, elderly sisters, elderly lady, elderly maid, elderly student, bedridden woman",
leidenGamma256.0 to leidenGamma512.0,"old couple, old woman, old lady, senior woman, older woman, old maid",,
leidenGamma512.0 to leidenGamma1024.0,"old couple, old woman, old lady","senior woman, older woman, old maid",


In [454]:
compare_gamma_for_theme("musician, musicians", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"acoustic guitar, air guitar, Music Students, Music student, maverick musician, Music fans, Era-defining artists, artistes locaux, street musician, Young performers, guitar player, Guitarist audition, guitarist, guitar virtuoso, guitar freaks, electric guitar, guitar, guitars, Street performances, street performer, street performers, up-and-coming artists, local artists, music artists, emerging talents, Singing lessons, contemporary artists, international artists, guitar craft, guitar lessons, Female artist, women artists, American musician, LA musician, cult musician, musician, musicians, Prominent artists, Old musician",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"Music Students, Music student, maverick musician, Music fans, Era-defining artists, artistes locaux, street musician, Young performers, Street performances, street performer, street performers, up-and-coming artists, local artists, music artists, emerging talents, contemporary artists, international artists, Female artist, women artists, American musician, LA musician, cult musician, musician, musicians, Prominent artists, Old musician","acoustic guitar, air guitar, guitar player, Guitarist audition, guitarist, guitar virtuoso, guitar freaks, electric guitar, guitar, guitars, Singing lessons, guitar craft, guitar lessons",
leidenGamma64.0 to leidenGamma128.0,"Music Students, Music student, maverick musician, Music fans, Era-defining artists, artistes locaux, street musician, Young performers, Street performances, street performer, street performers, up-and-coming artists, local artists, music artists, emerging talents, contemporary artists, international artists, Female artist, women artists, American musician, LA musician, cult musician, musician, musicians, Prominent artists, Old musician",,
leidenGamma128.0 to leidenGamma256.0,"Street performances, street performer, street performers, American musician, LA musician, cult musician, musician, musicians, Old musician, Music Students, Music student, maverick musician, Music fans, street musician","Era-defining artists, artistes locaux, Young performers, up-and-coming artists, local artists, music artists, emerging talents, contemporary artists, international artists, Female artist, women artists, Prominent artists",
leidenGamma256.0 to leidenGamma512.0,"American musician, LA musician, cult musician, musician, musicians, Old musician, Music Students, Music student, maverick musician","Street performances, street performer, street performers, Music fans, street musician",
leidenGamma512.0 to leidenGamma1024.0,"Music Students, Music student, maverick musician, Music fans, American musician, LA musician, cult musician, musician, musicians, Old musician",,Music fans


In [456]:
compare_gamma_for_theme("horse, horses", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,"horse dreams, horseback, real horse, horse, horses, Equestrian, white horses, pale white horse",NaN,NaN
leidenGamma32.0 to leidenGamma64.0,"horse dreams, horseback, real horse, horse, horses, Equestrian, white horses, pale white horse",,
leidenGamma64.0 to leidenGamma128.0,"horse dreams, horseback, real horse, horse, horses, Equestrian, white horses, pale white horse",,
leidenGamma128.0 to leidenGamma256.0,"horse dreams, horseback, real horse, horse, horses, Equestrian, white horses, pale white horse",,
leidenGamma256.0 to leidenGamma512.0,"horse dreams, horseback, real horse, horse, horses, Equestrian, white horses, pale white horse",,
leidenGamma512.0 to leidenGamma1024.0,"horse dreams, horseback, real horse, horse, horses, Equestrian, white horses, pale white horse",,


In [457]:
compare_gamma_for_theme("Starfish", [32.0, 64.0, 128.0, 256.0, 512.0, 1024.0])

,final_themes,dropped_themes,added_themes
leidenGamma32.0,Starfish,NaN,NaN
leidenGamma32.0 to leidenGamma64.0,Starfish,,
leidenGamma64.0 to leidenGamma128.0,Starfish,,
leidenGamma128.0 to leidenGamma256.0,Starfish,,
leidenGamma256.0 to leidenGamma512.0,Starfish,,
leidenGamma512.0 to leidenGamma1024.0,Starfish,,


# Write best Leiden communities to Neo4j
Based on the examples above, 256.0 seems like a good level for gamma.
Rename the property LeidenGamma32.0 from the graph projection to leidenId in the on-disk graph.
Drop any leidenId values from a previous run.

In [463]:
gds.run_cypher("MATCH (t:Theme|Stem) SET t.leidenId = null")
gds.graph.nodeProperties.write(g_themes2, {"leidenGamma512.0": "leidenId"})

writeMillis                       63
graphName            reweight_themes
nodeProperties            [leidenId]
propertiesWritten              19645
Name: 0, dtype: object

Create ThemeGroup nodes connecting themes that are duplicates.

In [464]:
gds.run_cypher("""CREATE CONSTRAINT theme_group_node_key IF NOT EXISTS FOR (n:ThemeGroup) REQUIRE n.id IS NODE KEY""")

""


In [472]:
gds.run_cypher("""
MATCH (t:Theme)
WHERE t.leidenId IS NOT NULL
WITH t ORDER BY t.description
WITH t.leidenId AS id, collect(t) AS themeNodes, collect(t.description) AS descriptions
MERGE (g:ThemeGroup {id:id}) 
SET g.descriptions = descriptions
WITH g, themeNodes
FOREACH (t in themeNodes | MERGE (t)-[:IN_GROUP]->(g))
""")

""


In [473]:
gds.run_cypher("""
MATCH (s:Stem)
WHERE s.leidenId IS NOT NULL
CALL
{
WITH s
MERGE (g:ThemeGroup {id:s.leidenId}) 
SET g.descriptions = coalesce(g.descriptions, []) + s.descriptions
WITH s, g
MATCH (s)<-[:HAS_STEM]-(t)
MERGE (t)-[:IN_GROUP]->(g)
}
""")

""


Create ThemeGroup nodes for Theme nodes that are not already grouped in a theme group if the theme is referenced in more than one document.

In [474]:
gds.run_cypher("""
MATCH (g:ThemeGroup)
WITH max(g.id) AS maxId
MATCH (t:Theme)
WHERE NOT EXISTS {(t)-[:IN_GROUP]->()}
AND COUNT {(t)<-[:HAS_THEME]-()}  > 1
WITH maxId, COLLECT(t) AS needsGroup
UNWIND range(0, size(needsGroup)-1) AS idx
WITH maxId, idx, needsGroup[idx] AS t
MERGE (g:ThemeGroup {id:maxId + idx + 1})
SET g.descriptions = [t.description]
WITH g, t
MERGE (t)-[:IN_GROUP]->(g)
RETURN COUNT(*) AS singleThemeGroups
""")

,singleThemeGroups
0,2891


For Theme nodes that are not already in a theme group and contain only one document, create ThemeGroup nodes and add a simple summary.

In [512]:
single_doc_df = gds.run_cypher("""
    MATCH (g:ThemeGroup)
    WITH max(g.id) AS maxId
    MATCH (t:Theme) WHERE NOT EXISTS {(t)-[:IN_GROUP]->()} 
    WITH maxId, COLLECT(t) AS needsGroup
    UNWIND range(0, size(needsGroup)-1) AS idx
    WITH maxId, idx, needsGroup[idx] AS t
    CALL
    {
    WITH t, maxId, idx
    MERGE (g:ThemeGroup {id:maxId + idx + 1})
    SET g.descriptions = [t.description]
    SET g.summary = "A movie about " + t.description
    MERGE (t)-[:IN_GROUP]->(g)
    RETURN g
    } IN TRANSACTIONS OF 10000 ROWS
    RETURN g.id AS groupId, g.summary AS summary""")

In [513]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key)

In [514]:
single_doc_df['embedding'] = embeddings.embed_documents(single_doc_df['summary'].tolist())

In [515]:
gds.run_cypher("""
UNWIND $data AS row
CALL
{
WITH row
MATCH (g:ThemeGroup {id:row['groupId']})
CALL db.create.setNodeVectorProperty(g, 'summaryEmbedding', row['embedding'])
} IN TRANSACTIONS OF 10000 ROWS
""", {"data": single_doc_df[['groupId', 'embedding']].to_dict("records")})

""


# Clean up projections

In [479]:
gds.graph.drop(g_themes)

In [480]:
gds.graph.drop(g_themes2)

# Add mean vector representation to each ThemeGroup

In [482]:
group_embeddings_df = gds.run_cypher("""
MATCH (g:ThemeGroup)<-[:IN_GROUP]-(t)
RETURN g.id AS groupId, collect(t.embedding) AS embeddings
""")

In [483]:
group_embeddings_df['mean_embedding'] = group_embeddings_df.apply(get_mean_vector, axis=1)

In [484]:
gds.run_cypher("""
    UNWIND $data AS row
    MATCH (g:ThemeGroup {id: row['groupId']})
    CALL db.create.setNodeVectorProperty(g, "meanEmbedding", row['mean_embedding'])""",
                   {"data":group_embeddings_df[['groupId', 'mean_embedding']].to_dict("records")})


""


In [485]:
gds.run_cypher("""CREATE VECTOR INDEX theme_group_mean_vectors IF NOT EXISTS 
                  FOR (t:ThemeGroup)
                  ON (t.meanEmbedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 1536,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""
